In [11]:
from keras.models import load_model
from tensorflow.keras.models import Sequential, model_from_json
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
# encoder model
# image feature layers
inputs1 = Input(shape=(4096,), name="image")
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
# sequence feature layers
inputs2 = Input(shape=(40,), name="text")
se1 = Embedding(8428, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

# decoder model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(8428, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.load_weights('/kaggle/input/best-model/best_model.h5')





In [17]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import random

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = self.load_data(file_path)

    def load_data(self, file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()
        data = []
        for line in lines:
            sentence_list = line.strip().split("\t")[1:]
            data.append(" ".join(sentence_list))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data[idx]
        inputs = self.tokenizer.encode(
            input_text, 
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return inputs
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss if labels is not None else None
        return loss


# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
# Load the dataset
dataset = MyDataset("/kaggle/input/allcaps/allcaps.txt", tokenizer, max_length=128)

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

from torch.nn.utils.rnn import pad_sequence

# Define a custom collator
from torch.nn.utils.rnn import pad_sequence
# Define a custom collator
# Define a custom collator
# Define a custom collator
def my_collator(features):
    input_ids = [feature[0] for feature in features]  # Extract the tensor from the list
    attention_mask = [[1] * len(ids) for ids in input_ids]

    # Convert attention_mask to tensors
    attention_mask_tensors = [torch.tensor(mask) for mask in attention_mask]

    # Pad sequences
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_attention_mask = pad_sequence(attention_mask_tensors, batch_first=True, padding_value=0)

    return {
        'input_ids': padded_input_ids,
        'attention_mask': padded_attention_mask
    }

# Instantiate the custom model
custom_model = CustomModel(model)

# Fine-tune the model
trainer = Trainer(
    model=custom_model,
    args=training_args,
    train_dataset=dataset,
    data_collator=my_collator
)

# Train the model
trainer.train()



# Save the fine-tuned model
# model.save_pretrained("fine_tuned_gpt2")
# tokenizer.save_pretrained("fine_tuned_gpt2")


TypeError: 'NoneType' object is not subscriptable

In [32]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
print("xy")
model = GPT2LMHeadModel.from_pretrained("gpt2-large")


xy


In [33]:
import torch
model.eval()
temperature = 3
top_k = 40
def generate_story(image_caption_pairs, tokenizer, model):
    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Set maximum length for generated text
    max_length = 100

    # Generate stories for each image-caption pair
    generated_stories = []
    for image_caption_pair in image_caption_pairs:
        # Tokenize and encode the image caption
        input_text = f" {image_caption_pair} "
        input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

        # Generate text based on the input
        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                max_length=max_length,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True,
                temperature=temperature,
                top_k=top_k,
                num_return_sequences=1,
                attention_mask=input_ids.ne(tokenizer.pad_token_id)  # Attention mask
            )

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_stories.append(generated_text)

    return generated_stories

# Example usage:
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token ID to eos_token_id
# Example usage:
image_caption_pairs = gen
generated_stories = generate_story(image_caption_pairs, tokenizer, model)
for i, story in enumerate(generated_stories):
    print(f"Generated Story {i+1}: {story}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generated Story 1:  a man wearing a hat is firing a gun in a city   To view this page ensure that Adobe Flash Player version 10.2.0 or greater is installed.

Police are investigating a shooting that left one man dead and two others wounded in the city's West Side neighborhood, police said.The shooting happened around 11:30 p.m. Sunday on the 5200 block of West Irving Park Road, according to Chicago Police News Affairs Officer Hector Alfaro.Police said

Generated Story 2:  a man is walking down a sidewalk with a bicycle in the background  

A man walks down the street with his bicycle on his shoulder. He is wearing a red shirt and blue jeans. A woman is standing next to him on the sidewalk. She is holding a sign that says, "I am a woman." The man looks at the woman and smiles. The woman smiles back at him. They continue on their way until they come to an intersection. At the intersection, there is a

Generated Story 3:  a smiling girl in a blue jacket and glasses is running in a wooded

In [20]:
# Concatenate image-caption pairs into input sequences
import torch
model.eval()
temperature = 3
top_k = 40
def generate_story(image_caption_pairs):
    # Load pretrained GPT-2 model and tokenizer
    tokenizer.pad_token = tokenizer.eos_token  # Set padding token to end-of-sequence token

    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(device)
    # Set maximum length for generated text
    max_length = 200

    # Generate stories for each image-caption pair
    generated_stories = []
    for image_caption_pair in image_caption_pairs:
        # Tokenize and encode the image caption
        input_text = f" {image_caption_pair} "
        input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

        # Generate text based on the input
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=max_length, num_beams = 5, no_repeat_ngram_size = 2, early_stopping = True, temperature=temperature, top_k=top_k, num_return_sequences=1)

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_stories.append(generated_text)

    return generated_stories

# Example usage:
image_caption_pairs = [
    'My name is Mansi and I am a LinkedIn Influencer',
    'A white girl is sitting on an office chair with pink top and specs watching terminator',
    'The boy went to the temple and said jai shree ram'
]

generated_stories = generate_story(image_caption_pairs)
for i, story in enumerate(generated_stories):
    print(f"Generated Story {i+1}: {story}\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


cuda


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Story 1:  My name is Mansi and I am a LinkedIn Influencer!!!

I am the founder and CEO of LinkedIn.com, a social media platform for people to connect with each other. I have been an active member of the LinkedIn community for over 10 years and have worked on a wide range of projects, including the development of social networking platforms like Facebook, Twitter, LinkedIn, Instagram, and Pinterest. My work has been featured in publications such as The New York Times, The Wall Street Journal, Business Insider, Forbes, Time Magazine, Fortune, USA Today, Newsweek, the New Yorker and many more. You can follow me on Twitter at @mansi.

Generated Story 2:  A white girl is sitting on an office chair with pink top and specs watching terminator ichthyosis.

"I don't know what you're talking about," she says. "I'm just trying to figure out what's going on. I can't tell you how many times I've been in the hospital, and I'm not sure what it's like to be in a hospital. It's not like I hav

In [17]:
#attention mask
import torch

# Concatenate image-caption pairs into input sequences
def generate_story(image_caption_pairs, tokenizer, model):
    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Set maximum length for generated text
    max_length = 50

    # Generate stories for each image-caption pair
    generated_stories = []
    for image_caption_pair in image_caption_pairs:
        # Tokenize and encode the image caption
        input_text = f"{image_caption_pair} "
        input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

        # Generate text based on the input
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature, top_k=top_k, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_stories.append(generated_text)

    return generated_stories

# Example usage:
image_caption_pairs = [
    'My name is Mansi and I am a LinkedIn Influencer',
    'A white girl is sitting on an office chair with pink top and specs watching terminator'
]

# Assuming you have already loaded the tokenizer and model

generated_stories = generate_story(image_caption_pairs, tokenizer, model)
for i, story in enumerate(generated_stories):
    print(f"Generated Story {i+1}: {story}\n")


AttributeError: 'ellipsis' object has no attribute 'to'

In [8]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Function to calculate entity grid
def calculate_entity_grid(doc):
    entity_grid = {}
    for sentence in doc.sents:
        entities = [ent.label_ for ent in sentence.ents]
        entity_grid[sentence] = entities
    return entity_grid

# Function to calculate lexical chains
def calculate_lexical_chains(doc):
    lexical_chains = []
    for token in doc:
        if token.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADJ']:  # Consider important parts of speech
            chain_found = False
            for chain in lexical_chains:
                if token.text.lower() in [word.lower() for word in chain]:
                    chain.append(token.text)
                    chain_found = True
                    break
            if not chain_found:
                lexical_chains.append([token.text])
    return lexical_chains

# Function to calculate coherence relations
def calculate_coherence_relations(doc):
    coherence_relations = []
    for sentence in doc.sents:
        coherence_relations.append(sentence.root.dep_)
    return coherence_relations

# Your paragraph
paragraph = "startseq a brown dog is sleeping on the grass watching star wars endseq  and then the next day the dog wakes up and goes to the park and plays with the other dogs."

# Process the paragraph using SpaCy
doc = nlp(paragraph)

# Calculate metrics
entity_grid = calculate_entity_grid(doc)
lexical_chains = calculate_lexical_chains(doc)
coherence_relations = calculate_coherence_relations(doc)

# Print results
print("Entity Grid:")
for sentence, entities in entity_grid.items():
    print(f"{sentence.text.strip()}: {entities}")

print("\nLexical Chains:")
for chain in lexical_chains:
    print(chain)

print("\nCoherence Relations:")
print(coherence_relations)


Entity Grid:
startseq a brown dog is sleeping on the grass watching star wars endseq  and then the next day the dog wakes up and goes to the park and plays with the other dogs.: ['DATE']

Lexical Chains:
['startseq']
['brown']
['dog', 'dog']
['sleeping']
['grass']
['watching']
['star']
['wars']
['endseq']
['next']
['day']
['wakes']
['goes']
['park']
['plays']
['other']
['dogs']

Coherence Relations:
['ROOT']


In [3]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize

# Sample documents
documents =   ['My hkjeabkr  euauhtuwaf jkauithawgtiu jnaw kgjnal']
# Tokenize documents
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]

# Create a dictionary representation of the documents
dictionary = Dictionary(tokenized_docs)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# Train an LDA model on the corpus
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary)

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print("Coherence Score:", coherence_lda)


Coherence Score: 1.0


In [14]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.downloader as api
import numpy as np

# Sample sentences
sentences = ['Typically, coherence scores are normalized or scaled in a way that makes them comparable across different models or parameter settings within the same experiment.',
             ' For example, in Gensims implementation, coherence scores usually range between -1 and 1, where higher values indicate better coherence. However, different coherence measures may use different scales or normalization techniques.']

# Tokenize sentences and remove stopwords
stop_words = set(stopwords.words('english'))
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]
tokenized_sentences = [[word for word in doc if word not in stop_words] for doc in tokenized_sentences]

# Create a dictionary representation of the sentences
dictionary = Dictionary(tokenized_sentences)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_sentences]

# Train an LDA model on the corpus
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary)

# Load Word2Vec model
word2vec_model = api.load("glove-wiki-gigaword-100")

# Compute coherence scores
coherence_cv = CoherenceModel(model=lda_model, texts=tokenized_sentences, dictionary=dictionary, coherence='c_v').get_coherence()
coherence_uci = CoherenceModel(model=lda_model, texts=tokenized_sentences, dictionary=dictionary, coherence='u_mass').get_coherence()
coherence_npmi = CoherenceModel(model=lda_model, texts=tokenized_sentences, dictionary=dictionary, coherence='c_npmi').get_coherence()

# Compute C_w2v coherence score
topic_words = [[dictionary[word_id] for word_id, _ in lda_model.get_topic_terms(topic_id, topn=10)] for topic_id in range(lda_model.num_topics)]
word_vectors = [[word2vec_model[word] for word in words if word in word2vec_model] for words in topic_words]
coherence_w2v = np.mean([np.mean([np.max([np.dot(word_vector1, word_vector2) for word_vector1 in vectors1]) for word_vector2 in vectors2]) for vectors1, vectors2 in zip(word_vectors, word_vectors[1:])])

print("C_v Coherence Score:", coherence_cv)
print("C_uci Coherence Score:", coherence_uci)
print("C_npmi Coherence Score:", coherence_npmi)
print("C_w2v Coherence Score:", coherence_w2v)


C_v Coherence Score: 0.644364630725974
C_uci Coherence Score: -2.8256198388331413
C_npmi Coherence Score: -0.15213270539559134
C_w2v Coherence Score: 23.014366


In [8]:
gen = ['a man wearing a hat is firing a gun in a city ',
 'a man is walking down a sidewalk with a bicycle in the background ',
 'a smiling girl in a blue jacket and glasses is running in a wooded area ',
 'a man wearing a black jacket and glasses looks down ',
 'a man with a brown and white and white jacket is holding a stick ',
 'a dog jumps over a hurdle ',
 'a soccer player in red is chasing a soccer ball ',
 'a man is standing on a beach with a man in a blue shirt and hat ',
 'a baseball player in a red shirt is trying to throw a baseball ',
 'a man in a red had is climbing a rock wall ',
 'a man in a black shirt and tie is talking to a man in a black shirt ',
 'a basketball player is jumping for the ball ',
 'a man talks on a cellphone while he talks on a cellphone ',
 'a group of people are gathered outside a frisbee ',
 'a man in a helmet is climbing up a rock ',
 'a snowboarder is jumping over a snowy hill ',
 'a girl in a blue shirt is jumping in a blue house ',
 'a man eats a dog while a woman eats ',
 'a brown dog is running in a field ',
 'a man is waterskiing ',
 'a man is standing on a brick bench with a big piece of lights ',
 'a man in a blue jacket is standing in front of a large tree ',
 'a brown dog jumping over a red obstacle ',
 'a man whose gear is submerged in water ',
 'a brown dog is running in the grass ',
 'a man and a woman are posing for a picture ',
 'a man shops for a picture of a market ',
 'a black and white dog is standing in front of a red and white stuffed animal ',
 'a man in a red jacket walks past a man in a red jacket ',
 'a man wearing a blue jacket and sunglasses is standing in front of a building ',
 'a man and a woman are embracing in front of a building ',
 'a man in a red jacket is walking through a busy street ',
 'a small dog is running down a bridge over a bridge ',
 'a small white dog is jumping over a pole ',
 'three boys are sitting on a wooden bench in front of a store ',
 'a boy swings on a swing ',
 'a man paddles a canoe down a river ',
 'a man sits on a bench in front of a wall with a bag ',
 'a snowboarder is performing a jump off a ramp ',
 'a group of children in a row ',
 'a man eats a slice of a slice of paper ',
 'a dog is in the snow ',
 'a man wearing a hat and a man in a white suit and a man in a white suit talking on a cellphone ',
 'a man is walking along a river with a fishing pole in it ',
 'a man is riding a motorcycle on a dirt track ',
 'a girl in a red coat and red coat is skiing ',
 'a man in a blue wetsuit rides a wave ',
 'two boys one with glasses kissing ',
 'a brown and white dog yawning ',
 'a german shepherd is laying on the grass by a man who is training him ',
 'a man and woman are walking around a coffee shop ',
 'a man in a blue shirt and a hooded shirt is skateboarding ',
 'a brown dog is running with a toy in its mouth ',
 'a woman in a blue shirt is holding a basketball ',
 'a brown and white dog running through grass ',
 'a man in a blue jacket and glasses is walking through a park ',
 'a woman and a woman are sitting on a bench in front of a house ',
 'a dog is chasing a man in a black jacket ',
 'a man in a red shirt is walking past a man in a red shirt ',
 'a backpacker poses for a picture ',
 'a man is holding a ball on a skateboard ',
 'a man in a red jacket and a woman in a red jacket talking on a tripod ',
 'a man sits on a rock formation ',
 'a basketball player is being thrown by the two men ',
 'a person on a motorcyle is airborne ',
 'a man is walking on a snowy hill ',
 'a brown dog is running through a field of water ',
 'a man and a man are posing for a picture ',
 'a man is walking on a snowboard in a skate park ',
 'a man rows a boat ',
 'a man is climbing a rock ',
 'a man in a brown sweater is holding a funny cheeks ',
 'a man and a man are making a funny face for a picture ',
 'a man and a woman sit on a rock wall ',
 'a girl with a nylon ribbon is blowing bubbles ',
 'two men are sitting on canoes in canoes ',
 'a black dog carries a buoy in the snow ',
 'a black and white dog is walking through a snowy area ',
 'a man and a man are standing in front of a wall ',
 'a man wearing a red shirt is skating along a concrete ledge ',
 'a dog jumps over a log in a snowy field ',
 'a little boy in a blue shirt is in a blue shirt ',
 'a football player in red and white is running in a game ',
 'a man is sitting on a rock under a tree ',
 'a brown dog is running in a field of flowers ',
 'a dog catches a ball in his nose ',
 'a man wearing a white shirt is jumping up into the air ',
 'three dogs run through the water ',
 'a dog chases a ball in a field ',
 'a dog is biting a stick from a wooden object in its mouth ',
 'a man is jumping off a red and red jump ',
 'a brown dog is running through a field ',
 'a man in a white tracksuit is walking barefoot in a gym ',
 'a man in a red jacket is walking down a busy street ',
 'a man pushes a wagon with a toy written in her hand ',
 'a girl is fixing a red and red bicycle ',
 'a man and a dog are jumping over a pole ',
 'a black dog with a red collar is walking through the water ',
 'a man in a red jacket and a red hardhat is wearing a red helmet ',
 'a man walks down a sidewalk in a park ',
 'a skateboarder is doing a trick in the air ',
 'a white bird is flying over a white and white body of water ',
 'a man in a red shirt is walking down a street with a fence in the background ',
 'a man in a green jacket is walking through a busy city street ',
 'a man in a blue shirt is walking on a dirt road ',
 'a man rides a bike through a forest ',
 'a girl in a red shirt and a flowery shirt is observed by a woman in a red shirt ',
 'a group of people are backpacking through a race ',
 'a man in a swimming pool flipping up to emerge from a pool ',
 'a man in a flannel shirt is skating on a skateboard ',
 'a man in a white jacket and a beard is holding a microphone while another man is holding a microphone ',
 'a white bird flies through the water ',
 'two dogs run through a field ',
 'a white bird grazes the water ',
 'a dog holds a ball in its mouth ',
 'a black and white dog is jumping over a red frisbee ',
 'a man in a black shirt is jumping into a pool ',
 'a girl in a ridable shirt listens to a ride ',
 'a woman in a black shirt and a woman in a red dress ',
 'a man and a woman are kissing a younger girl ',
 'a man is walking past a bear shop ',
 'a man in a yellow life jacket rides a wave ',
 'a girl in a blue shirt is smiling while playing on a swing ',
 'a motorcyclist on a motorcycle ',
 'a man is walking down a sidewalk with a segway and a red and red umbrella ',
 'a dog is running through a grassy field ',
 'a dog jumping over a hurdle ',
 'a man in a red shirt is walking past a man in a red shirt and a backpack ',
 'a man in a red jacket is starring at something in the distance ',
 'a man wearing a red and red uniform riding a bike on a street ',
 'a dog is running with a yellow toy ',
 'a man is riding a bike on a hill ',
 'a black dog is walking down a frozen ice ',
 'a small black and white dog is running with a red ball in its mouth ',
 'a man in a black shirt and black shirt is sitting on a wall ',
 'a man is jumping up into the sand at the beach ',
 'three shirtless shirtless swimmers in murky murky murky murky murky murky murky murky green water ',
 'a boy is jumping his back into a fence ',
 'a young girl peering down a tree ',
 'a dog is chasing a small dog that is chasing a ball ',
 'a man and a woman shop for a record displaying a record ',
 'a baseball player in a white uniform is swinging a bat ',
 'a brown and white dog is jumping over a red frisbee ',
 'a biker is jumping over a ramp ',
 'a man in a red jacket and sunglasses is smoking a cigarette ',
 'a boy hangs off a pole ',
 'a black dog is chasing a black dog through the grass ',
 'a cyclist is doing a jump on a bike ',
 'a boy is pushing a boy who is pushing him as he is pushing him ',
 'a dog is jumping into the water ',
 'a man in a black shirt and a woman riding a bicycle in the street ',
 'a determined girl in a white and white determined is playing soccer ',
 'a girl in a red jacket and sandals is walking on a sidewalk ',
 'a boy in a blue life jacket is holding a backstroke in a swimming pool ',
 'a man in a red sweatshirt and a red sweatshirt is walking through a city street ',
 'a man is sitting in a basketball with a basketball ',
 'a white dog is running through a field of grass ',
 'a girl in a bathing suit is walking on the beach ',
 'a dog jumping over a hurdle ',
 'two soccer players are chasing after a soccer ball ',
 'a dog is jumping over a striped and white bar ',
 'a man in a blue jacket is standing in front of a window ',
 'a man is poised to catch a frisbee in a field ',
 'a man is bending down on a chain ',
 'a man holding a pirate and a baseball cap on a chair ',
 'a dog runs through the snow ',
 'a man in a red jacket and a man in a red jacket ',
 'a bug with a bug on his finger ',
 'a man with a backpack is walking through the snow ',
 'a boy in a blue shirt and jean hat is covered in a red bonnet ',
 'a little girl in a red shirt is ridding a girl ridding a girl ridding a little girl in a red dappled boat ',
 'a man in a white shirt and white pants is walking on a grassy field ',
 'a dog is swimming in the water ',
 'a man in a trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench trench walk stand in a bleak street ',
 'a man rides a bike down a hill ',
 'a brown dog is running through a grassy field ',
 'a surfer is falling off a surfboard ',
 'a brown dog is running through a grassy field ',
 'a black and white dog is running through a grassy field ',
 'a boy in a blue shirt is reaching from a wooden structure ',
 'two children are walking through a shallow puddle of water ',
 'a boy in a red shirt is riding a red and red roller coaster ',
 'a dog jumps over a pole ',
 'a girl in a red sweater and a woman in a red jacket ',
 'a black dog springs up to attack something ',
 'three boys are skating on a rink in front of a skating rink ',
 'a man in a white shirt is attacking a football player in a black jersey ',
 'a man flipped into the water ',
 'a woman in a white shirt and a white shirt is standing in front of a wedding ',
 'a brown dog is carrying a brown and white ball ',
 'a man is jumping up to catch a ball ',
 'a boy in a red shirt is smiling while another eats ',
 'a man in a red shirt and a woman in a red shirt ',
 'a woman and a woman are walking past a row of leaves ',
 'a brown and white dog is running through a grassy field ',
 'a football player is being tackled by the rival team ',
 'a woman with a baby in a chair ',
 'a man and a woman are standing on a wood floor ',
 'a man in a blue shirt is walking a scooter in a crowd ',
 'a woman palm his hair as he is reaching into a window ',
 'ever since his baby started to the arcade ',
 'a man and a woman are walking down a beach ',
 'a man in a red jacket is riding a bike through the woods ',
 'a group of people in a white shirt and a man in a white shirt ',
 'bridal wedding party bridesmaids in formal attire ',
 'a man wearing a black shirt is holding a baby hand while another boy looks on ',
 'a brown dog is laying on its back in a grassy pool ',
 'a man is riding a surfboard on a wave ',
 'a smiling girl eats with bread in a baby eating eating ',
 'a man wearing a white jacket and white hat is holding a white baseball ',
 'a man in a red jacket is kayaking in rough waters ',
 'a man in a red jacket is walking down a street with a saw ',
 'a man is climbing a rock wall ',
 'a white and white dog runs across a green field ',
 'a man in a red shirt is adjusting a red and red flag ',
 'a man is doing a bicycle stunt on a half pipe ',
 'a brown and white dog is running through a grassy yard ',
 'a woman in a bathing suit holds a baby in a black and yellow shirt ',
 'a group of asian asian men are standing in front of a brick wall ',
 'a boy is walking down a tree branch ',
 'a man in a black jacket and a woman in a subway ',
 'a man is holding a stick while holding a stick in his mouth ',
 'a man is steering a man in a conoe and a logo astride a man on a foggy foggy day ',
 'a little girl is playing soccer ',
 'a basketball player in a white uniform raises his fist ',
 'a baby in a diaper jumps onto a bed ',
 'a group of people are rafting down a choppy river in a field ',
 'a wet dog is walking through water ',
 'a dog is jumping over a red and white ball ',
 'a group of people are in a race car through a turn ',
 'a man juggling a man who is training it ',
 'a boy in a blue sweatshirt and glasses is swinging a rope ',
 'a man stands on a ledge at sunset ',
 'a brown dog is chasing after a brown and brown dog ',
 'two dogs play in the snow ',
 'a man sits on a sidewalk in front of a building ',
 'a boy in a red jacket is playing with bubbles ',
 'a boy is riding a tire swing ',
 'a boy is climbing a red rail ',
 'a man sits on a bench with a flag in a white shirt ',
 'a man in a black shirt and a man in a boat ',
 'a dog is running through a field ',
 'four people are walking down a moutain stream ',
 'a black dog chases a black dog in the grass ',
 'a man in a red shirt is climbing a rock wall ',
 'a man in a red jacket is standing in front of a restaurant counter ',
 'a person wearing a white hat and a beard is resting his face over a tree ',
 'a man and woman are backlit guitar ',
 'a man and a woman are sitting on a bench outside a window ',
 'a man is sitting on a bench in a darkened area ',
 'a man in a brown shirt is holding a baby in a cardboard restaurant ',
 'a boy runs through a grassy field ',
 'a boy in a yellow shirt is walking on the beach ',
 'a white dog is jumping over a white and white ball ',
 'a man plays a guitar while another man is holding a cigarette ',
 'a man in a red shirt is standing on a rock ',
 'a man wearing a white shirt and white cap is holding a tennis racket in his hand ',
 'a dog is jumping over a cylindrical a rock ',
 'a black dog jumps over a frisbee in the snow ',
 'a young girl with lips paint and lips paint ',
 'a man in a red jacket is standing in front of a large rock formation ',
 'a man is firing a gun in a bullseye attempting a very thrown from a painted bow ',
 'a black dog is running through the water ',
 'three dogs play in the snow ',
 'a skateboarder is jumping through the air ',
 'a skateboarder is jumping off a low ramp ',
 'a man in a black jacket walks down a busy street ',
 'a person is doing a trick on a bike on a bike ',
 'a man in a red jacket interviewing a man in a red jacket ',
 'a man in a blue jacket is fishing in the water ',
 'a black dog is jumping into a lack into a lack ',
 'a man is bending back to a pony while people are standing behind ',
 'a man climbing a rock wall ',
 'a man is tossing a man into a group of people ',
 'a brown dog is running through a grassy field ',
 'a brown and white dog is running through a grassy field ',
 'a dog runs through the sand ',
 'a boy is swinging on a rope swing ',
 'two boys are playing on a grassy field ',
 'a group of people are sitting in a mexican restaurant restaurant ',
 'a man in a black jacket is jumping in the snow ',
 'a group of people are gathered around a wall ',
 'a person jogs down a hill ',
 'a man in a black shirt and a black beard with a backpack on his head ',
 'a girl in a bathing suit is walking on a beach with a bucket of water ',
 'a man in a black shirt is walking past a man in black ',
 'a girl in a red and white uniform is chasing a soccer ball ',
 'a girl in a red snowsuit is sitting on a red blanket ',
 'a man and a woman are posing for a picture ',
 'a man is climbing a rock ',
 'a man is doing a trick on a snowboard in the snow ',
 'a brown dog is running through a rough field ',
 'a man with a backpack sits on a rock ',
 'a boy is holding a net to a guide ',
 'a baseball player recoils from the base of a baseball player in a white shirt ',
 'a man with a headwrap with a woman in a red headscarf ',
 'three dogs race around a track ',
 'a hockey player in black untangles from the puck for the puck ',
 'a skier is skiing down a snowy hill ',
 'a dog is running through snow ',
 'a little boy in a blue shirt is smiling while posing for a picture ',
 'three men are kayaking down a lake ',
 'a girl in a black and white dress is posing for a picture ',
 'a boy in a red uniform is holding a football ',
 'a man is doing a trick on a skateboard on a skateboard ',
 'a dog swimming in the water ',
 'a man falling off of strong waves in the ocean ',
 'three dogs are playing together in a field ',
 'a basketball player in a white uniform is dribbling the basketball ',
 'a woman in a black skirt and black boots walks down the sidewalk ',
 'a man in a suit and a man in a suit and tie kissing a man in a black shirt ',
 'two dogs play in a field ',
 'a man and a man are talking to use machines ',
 'a man and a woman are sitting on a bench in front of a lake ',
 'a man and a woman are walking down a rocky path ',
 'a boy in a dark shirt is sitting on a pile of rocks ',
 'a little girl is climbing onto a metal railing ',
 'a brown and white dog is jumping over a small white dog ',
 'a dog is jumping up to catch a ball in the grass ',
 'a man is playing fetch with a football ',
 'a brown dog is running through a grassy field ',
 'a man pushes a infant on a tricycle ',
 'a little girl is sitting on a red plastic toy ',
 'a woman in a white shirt and white pants is swinging a softball ',
 'a girl is playing with a piggy electric electric guitar ',
 'a man is walking past a building with a building in the background ',
 'a football layer in action ',
 'a man and woman are standing on a swing ',
 'a brown horse is running through a grassy field ',
 'a man is standing on a mountaintop ',
 'a dog jumps over a hurdle ',
 'a dog lies on the ground and bending straight in his mouth ',
 'a man is standing in front of a windmill ',
 'a man in a red shirt and a black and white jersey is running on a field ',
 'a black dog chasing after a frisbee ',
 'a girl in a black shirt is talking to a girl in a black shirt ',
 'a group of children in front of a structure ',
 'a dog is jumping over a wooden fence ',
 'a man in a blue shirt is cooking asleep on a stove ',
 'a little girl is playing with a stuffed animal ',
 'a man in a red shirt is reaching from a couch ',
 'a young girl is running through a field with a balloon in the background ',
 'a young boy wearing a red shirt is jogging on a sidewalk ',
 'a mule containig a calf is being chased by a cow ',
 'a black dog is running on the beach ',
 'a man and a woman are posing for a picture ',
 'a black dog is jumping over a white poodle who is jumping over a fence ',
 'a dog jumps to catch a frisbee ',
 'a brown dog is running through a grassy field ',
 'a man in a blue jacket stands in front of a store with a closeout ',
 'a man with a beard and a beard is sitting on a wooden wall ',
 'a woman in a white shirt is swinging a softball ',
 'a man with a black and white jacket is skiing ',
 'a man in a red jacket and a red helmet is riding a motorcycle ',
 'a man in a blue shirt and a man asking for money ',
 'a brindle coated dog is playing tug of war with a red and yellow toy ',
 'a girl in a bathing suit is sitting on a ledge with a balloon in his hand ',
 'a black dog is leaping in the air to catch a ball ',
 'three dogs play in the grass ',
 'a small girl ascends a large metal fountain ',
 'a woman in a white shirt holds a tennis racket while a cameraman looks turns ',
 'a black and white dog is running on the sand ',
 'a boy in a red jacket is swinging a bat ',
 'a girl in a red shirt and a stripe on a red and white stripe and a stripe in a red shirt ',
 'a young boy is playing with a red and white toy ',
 'a white dog is walking through snow ',
 'a man in a shirt is climbing a rock wall ',
 'a girl in a pink shirt and a girl on a swing ',
 'a man in red jackets is flipped down in the water ',
 'a man wearing a black jacket and sunglasses talks to a man in a black jacket ',
 'a man is jumping a man on a ropes while another helps him ',
 'a man with a matching mohawk and sunglasses smiles ',
 'a girl is sitting on a bench in front of a window ',
 'a woman in a black shirt is walking a black and white woman and a woman in a black shirt ',
 'a baby holds a red and white soccer ball ',
 'a woman in a black shirt and tattoos is starring off ',
 'a boy in a yellow shirt and a yellow shirt is jumping on a swing ',
 'a little girl is sitting on a wooden floor with his hand on his face ',
 'a man with lots of lots of plants is sewing ',
 'a cricket player in a white shirt is swinging a cricket bat ',
 'a woman in a tracksuit is walking down a wood stage ',
 'a brown dog is staring at a brown dog ',
 'a white dog is jumping over a red and white ball ',
 'a girl in a teal shirt smiles ',
 'a man in a werewolf suit is holding a camera ',
 'a group of people are standing around a burning burning building ',
 'a girl in a floral shirt is climbing a railing with a girl in a red shirt ',
 'a woman in a red jacket is walking down a sidewalk ',
 'a girl stands on a shore and looks out into the water ',
 'a boy is running on a grassy field with a balloon barrel in the background ',
 'a dog is chasing a soccer ball in the air ',
 'a boy in a blue swimsuit is playing in a pool ',
 'a white dog is jumping over a hose ',
 'a little girl is jumping over a multicolored inflatable and blue pole ',
 'a military dog jumps for a ball ',
 'a girl in a red shirt is smiling while a girl in a red shirt is smiling ',
 'a brown dog is jumping into the water ',
 'a dog jumps over a pole ',
 'a girl and a woman sit on a bench ',
 'a ragged man sleeping on a chair ',
 'a dog chases a ball in a grassy field ',
 'a group of people are walking through a circle ',
 'a man and woman walk along a beach ',
 'a man is sitting on a wooden bench beside a fence ',
 'a man in a floral sweater is wearing a floral bathroom ',
 'a skateboarder is jumping over a rail ',
 'a man is performing a trick on a skateboard on a wooden wall beside a graffiti covered wall ',
 'a man wearing a red jacket is wearing a pink jacket ',
 'a white dog is jumping over a hurdle ',
 'a german shepherd is carrying a stick in its mouth ',
 'a helicopter is flying through the air with a frisbee in the background ',
 'a man is talking on a cellphone in front of a tropical window ',
 'a girl in a red shirt is walking on a sidewalk ',
 'two dogs are playing together on the grass ',
 'a dog jumps over a couch ',
 'a man with a goatee is wearing a white shirt and a white shirt and a white shirt ',
 'a dog jumping over a hurdle ',
 'a dog is joined through a harness while a dog follows ',
 'a white dog is walking through the water ',
 'two dogs are playing in the snow ',
 'a brown dog sits on a blue lap ',
 'a man is standing on a rock with a rope ',
 'a young boy wearing a blue shirt and pirates ',
 'a man is walking past a cement wall with a red writing ',
 'a man sits on a rock ',
 'a man and woman are sitting around a table in front of a wall ',
 'a man in a black jacket is gazing through the snow ',
 'a bird flies low to the ground ',
 'a man in a blue shirt is skating on a skateboard ',
 'a man is jumping a man back to his birds ',
 'a man and a woman pose for a picture ',
 'a snowboarder is jumping over a snowy hill ',
 'a dog swimming with a tennis ball in its mouth ',
 'a woman is wearing a white veil and a sweater is sitting on a couch ',
 'two hockey players in a struggle on the ice ',
 'a man stands on a rock ',
 'a man walks through a city street ',
 'a man in a blue shirt is walking on a sidewalk with a man in a blue shirt ',
 'a white bird is flying over a landing ',
 'a skier is skiing down a snowy hill ',
 'a woman wearing sunglasses and a blue shirt is smiling ',
 'a group of children standing in front of a tree ',
 'a girl is sitting on a beach ',
 'a girl in a red uniform is clutching a football ',
 'a man in a black sweatshirt and a black sweatshirt is walking past a tv in a crowd ',
 'a man wearing a black shirt and a black shirt with a baby in a shirt sitting on a bus ',
 'a dog is jumping in the air to catch a frisbee ',
 'a man is doing a trick on his skateboard on a cement ledge ',
 'a man with a brazilian flag and a man in a yellow shirt and a flag in a pool ',
 'a group of people are walking through a rural area ',
 'a black dog is running with a red object in its mouth ',
 'a woman in a red shirt and a black shirt sitting on a picnic table with a plate in her hand ',
 'a woman holds a baby that is reaching for a baby ',
 'three children stand on a path by the beach ',
 'a man wearing a ponytail is holding a baby in a blue shirt ',
 'a boy is swimming in a pool ',
 'a man in a blue shirt is walking on a beach with a man in a hat and hat ',
 'a surfer rides a wave ',
 'a fluffy white dog is running through the snow ',
 'a man in a white uniform is dribbling a basketball ',
 'three dogs are running through the snow ',
 'a snowboarder is going down a snowy hill ',
 'a black dog is running through the snow ',
 'a baby with a dads nose ',
 'a black dog is running in the sand ',
 'a baseball player in a white shirt is getting ready to throw a ball ',
 'a young girl is soaking water ',
 'a man and a man are fencing ',
 'a dog jumps over a log in a lake ',
 'a dog is jumping over a rocky barrior ',
 'a man is doing a jump while another falls on a ski ladder ',
 'a man is sitting on a couch under a tree ',
 'a man muscular man is kissing a man in a black shirt ',
 'a girl in a blue shirt is hanging onto a wooden fence ',
 'a brown dog is running through a grassy field ',
 'a man is falling into the water ',
 'a man wearing sunglasses and a black shirt smiles for the camera ',
 'a man is on a bike in the woods ',
 'a german shepherd is running through the grass ',
 'a man is being splashed by water ',
 'a brown dog is jumping over a yellow and yellow toy ',
 'a man in a yellow jacket is sitting on a mountain ',
 'a bicyclist is jumping his bike in the air ',
 'a group of children are standing on a bridge in front of a lake ',
 'a woman in a white shirt and black pants is running on a track ',
 'a brown dog swimming in the water ',
 'three women in saris are practicing dancing on a wooden wall ',
 'a man in a black gown stands with a baby in a red shirt ',
 'a dog is chasing a yellow and yellow ball ',
 'a man wearing a red hat is climbing a rocky mountain ',
 'a man in a white shirt raises his arm around a man in a white and white suit ',
 'a girl in a brown shirt and a woman in a bicycle pose with a baby in a brown and red shirt ',
 'a man in a white shirt is surfing ',
 'a dog is walking along a foggy river ',
 'a girl in a red shirt is standing in a red plastic balloon house ',
 'a man and a girl are walking on the beach ',
 'three girls in bikinis are walking on the beach ',
 'a man and woman are sitting on a wood floor in front of a wall ',
 'a man wearing a red shirt is climbing a waterfall ',
 'a man wearing a hat and a hat talks on a cellphone ',
 'a young girl wearing a snorkeling cap and goggles gives a thumbs up sign ',
 'a woman in a red jacket and a woman in a red headscarf ',
 'a man and a woman are walking down a street with a lot of people ',
 'a man in a white shirt is bouncing on a karate bed ',
 'a dog swimming in water ',
 'a brown dog is bending on to bite another dog ',
 'a man with a camera and eyebrows talks on his cellphone ',
 'a man is walking down a road in a park ',
 'a man in a white shirt and black pants is chasing a soccer ball in a soccer game ',
 'a man is jumping up in the air while being watched by a man in a black shirt ',
 'two dogs run side in the grass ',
 'a dog is being chased by a dog in a race ',
 'a dog jumps over a hurdle ',
 'a group of women dressed in various costumes ',
 'a vase of a woman cutting a baby ',
 'a hockey player in a black and black uniform is trying to hit a ball ',
 'a man in a white shirt is carrying a basketball in his hand ',
 'a man in a red shirt and a red mohawk talks to a man in a red shirt ',
 'a man is sitting on a rocky path ',
 'a dog growling at a cow ',
 'a football player in a sooners uniform is preparing for the snap ',
 'a black dog is running in a grassy yard ',
 'a little boy in a red shirt is climbing onto a red slide ',
 'a group of people are sitting on a bench in front of a row of benches ',
 'a boy in a red shirt is kayaking in the water ',
 'a dog is jumping over a chain ',
 'a young boy is riding a tire in a dusty street ',
 'a man leads a dog to zigzag over poles ',
 'a skateboarder is doing a jump on a skateboard ',
 'a man is riding a dirt bike on a dirt course ',
 'a woman in a red shirt and black shirt is talking on a cellphone ',
 'a puppy is playing with a soccer ball ',
 'a girl in a red shirt has his arm around her eyes ',
 'a brown dog is walking on a leash in front of a tree ',
 'a black dog is measured over a black dog in a far obstacle course ',
 'a dog is jumping out of the ocean water ',
 'three naked girls are playing in a river ',
 'a soccer player in a white uniform is trying to catch a soccer ball ',
 'a man is practicing a dog on a grassy field ',
 'a man is sitting on a bench in a race ',
 'a black and white dog is running through a grassy field ',
 'a small dog is eating a small cat ',
 'a dog is jumping over a chain ',
 'a man in a red jacket is talking to a man in a red coat ',
 'a black dog is jumping over a red and white ball ',
 'a man is doing a trick on his skateboard ',
 'a man and woman have their baby and matching shirts ',
 'a man in a leather jacket looks down ',
 'a little girl is sitting on a wooden bench with a bucket of people in the background ',
 'a man in a red shirt is sitting on a bench with a basketball in the background ',
 'a group of girls are playing soccer on a field ',
 'a man is fishing in a lake ',
 'a man with a beard resting a wall with a hand resting his hand ',
 'a girl is jumping into the air on a beach ',
 'a man training a man who is training a man who is training a man who is training a man who is training a man who is training him ',
 'a man is rollerskating on a ramp with a skyline in it ',
 'a black dog is running on a beach with a ball in its mouth ',
 'a woman in a white shirt and blue shorts swings a tennis racket ',
 'a group of people are sitting on benches in front of a large ship ',
 'a man in a white jacket and sunglasses talks to a man in a white jacket ',
 'a little girl in a blue shirt is climbing on a wooden bench ',
 'a girl in swim trunks dives into the water ',
 'a woman wearing sunglasses and a mohawk is smiling ',
 'a boy is swinging on a swing ',
 'a young boy sitting on the ground in the forest ',
 'a brown and white dog running through a field ',
 'a white dog with a white tutu running in the sand ',
 'a boy with a backpack is walking on a sunny day ',
 'a young boy wearing a blue shirt pushing a baby swing ',
 'a black and white dog is jumping over a log in a snowy pool ',
 'a dog is standing in the snow ',
 'a car is making a jump in the air ',
 'a tennis player in a white uniform is measured by the tennis racket ',
 'a woman in a red jacket is walking through a city street ',
 'three children are standing on the beach by the ocean ',
 'a girl in a yellow shirt and yellow pants is jumping in the air ',
 'a little girl in a black shirt is sitting on a slide ',
 'a group of people are sitting on a ledge in front of a large crowd ',
 'two children are walking down a wet path ',
 'a man is flying kites in the sand ',
 'a dog jumps over a pole ',
 'a man in a red uniform is walking down a dirt game ',
 'a person is riding a snowboard on a beach ',
 'a snowboarder jumps off a hill ',
 'a man in a red uniform talks to a man in a red uniform ',
 'a closeup of a hummingbird ',
 'a man wearing a black jacket is jumping up to catch a yellow and yellow ball ',
 'a group of people are riding a dirt atv in a ditch ',
 'a man in a yellow boat cuts down a wave ',
 'a man and woman walk through the street ',
 'a man in a blue jacket is walking past a man with a red handbag ',
 'a man sits alone at a table reading a book ',
 'three girls in a red and orange coat ',
 'a dog is running through the water ',
 'a woman in a red shirt is walking along a busy beach ',
 'a dog bites a nap ',
 'a dog jumps over a stream ',
 'a man with a red hardhat is rock climbing ',
 'a brown and white dog is running on a grassy lawn ',
 'a hiker is walking on a snowy hill ',
 'a black dog is running on the sand ',
 'trainer is training her dog to get a ball ',
 'a man in a red shirt is standing in front of a building ',
 'a young boy in a red shirt climbing a tree ',
 'a man in a red jacket and black jacket walks down a sidewalk ',
 'a brown dog is jumping over a white and white dog ',
 'a young boy is playing with a small baby in a blue shirt ',
 'a boy in a blue shirt is blowing a bubble ',
 'a dog jumps over a chain ',
 'a man in sunglasses is wearing sunglasses ',
 'a man and a dog are jumping on a wooden deck ',
 'a man in a red hat and a red headwrap with a man in a red headscarf ',
 'a young boy is laughing while laughing ',
 'a man is paddling a canoe on a beach ',
 'three children swing on swings ',
 'a boy in a red shirt is jumping in the air ',
 'a man wearing a scarf smiles ',
 'a man is rock climbing ',
 'a girl in a pink dress is smiling whilst holding a dog ',
 'a dog jumps to catch a frisbee ',
 'a man with a fishing pole is walking on a rocky shore ',
 'a man and a woman are walking past a billboard ',
 'a man and a dog are jumping over a wooden fence ',
 'a white dog is jumping over a log in the snow ',
 'a dog swimming in the water ',
 'a dog digs in the snow ',
 'a dalmation is wearing a black and white shirt ',
 'a white bird lifts it landing in the water ',
 'a man and a woman are looking at something in the distance ',
 'a snowboarder in the snow ',
 'a skateboarder rides a basketball ',
 'a man in a helmet and a beard pushing a bicycle ',
 'a woman in a blue skirt and blue skirt is walking down a street with diners in the background ',
 'a small black and brown dog is playing with a ball ',
 'a baseball player in a baseball uniform tags a baseball ',
 'a man in a white shirt is jumping in the air to catch a frisbee ',
 'a man in a red jacket is walking through a red race ',
 'a woman in a skimpy dress and tight boots is walking away from a woman in skimpy clothing ',
 'a man in a white shirt and white pants is playing soccer ',
 'marching shop in front of a fence ',
 'a woman in a suit and jacket talks on a cellphone ',
 'a dog is jumping in the air to catch a frisbee ',
 'a boy in a bathing suit is getting splashed by a fountain ',
 'a man rides a bike on a dirt path ',
 'a black and white dog is playing with a toy in a rocky area ',
 'a man and a woman sit on a bench ',
 'a skateboarder is performing a jump on a wooden bench ',
 'a black dog is running through grass ',
 'a man in a red shirt is standing in a body of water ',
 'a man and a woman are walking on a park bench ',
 'a man is doing a trick on a bike in a skatepark ',
 'a white bird is walking through the snow ',
 'three girls are standing on a red and red sculpture ',
 'a man wearing a hat and a hat is sitting on a bench with a man in a blue jacket ',
 'a woman in a red sweater is smiling with a girl in a red sweater ',
 'a small black and white dog is running in the grass ',
 'a dog is standing in the snow ',
 'a brown dog is jumping over a brown dog ',
 'a dog is running through a grassy field ',
 'a man and a woman are posing for a picture ',
 'a young boy is playing with a white and white bat ',
 'a dog is jumping over a snowball ',
 'a white dog is jumping in the air to catch a soccer ball ',
 'a man in a red hat is holding a red and red hat and carrying a red beverage ',
 'four children are playing in a knight and royal animal scenery ',
 'a man is being tackled by another football player in a football game ',
 'a man in a blue jacket talks to a man in a blue jacket ',
 'a girl in a red shirt is walking through a crowd of people ',
 'a town of a town ',
 'a young boy in a blue shirt and glasses is smiling ',
 'a cubby cheeked baby in a red bib with a baby in a red shirt ',
 'two dogs confront each other in a field ',
 'three boys are jumping into a swimming pool ',
 'a man wearing a white jacket and beige hat talks to another man ',
 'a boy in a red shirt and tie is pointing down a wooden wall ',
 'a young boy is jumping on a beach ',
 'a group of children playing in a grassy yard ',
 'a girl in a floral shirt and a woman in a red shirt ',
 'a man and a woman are walking on a beach ',
 'a dog is running down a grassy path ',
 'a girl in a polka suit dances on a trampoline ',
 'a man is standing on a barrel with a rainbow and a rainbow moon behind him ',
 'a baby in a red shirt has a blue figure in a blue playpen ',
 'a man in a red cap and a man in a red shirt are posing for a picture ',
 'a man is sitting on a rock bench ',
 'a man is surfing a surfboard ',
 'a man in a red coat is crossing a street with a police poster ',
 'a man is kayaking on a lake ',
 'a brown and white dog is running through the snow ',
 'a man rides a bike down a dirt path ',
 'a woman in a red shirt and a black shirt and a black shirt and a woman in a red dress ',
 'a man and a woman sit outside a table ',
 'a dog jumps over a couch ',
 'a dog in a harness is being held by a dog ',
 'a tractor decorated truck ',
 'a man and a woman pose for a picture ',
 'a little boy in a blue shirt is hanging onto a multicolored structure on a playground ',
 'a man climbs a steep rock wall ',
 'a boy with a toddler face and face paint is holding a baby who is using a funny face ',
 'a dog swimming in the water ',
 'a woman in a black shirt and a woman in a black shirt standing in front of a lake ',
 'a man in a red and white outfit is walking on a field ',
 'a man in a red jacket interviewing a man in a crowd of people ',
 'a skateboarder does a trick on a ramp ',
 'a white dog is running on the grass ',
 'a dog is jumping over a log in a forest ',
 'a dog is running through a grassy field ',
 'a brown dog is running through a green field ',
 'three people sitting at a table with a food in front of them ',
 'a black dog is jumping over a snowy hill ',
 'a boy is splashed by water situated on a water slide ',
 'a white dog with a yellow collar is retrieving a stick ',
 'a girl in a white shirt is sitting on a rock ',
 'a man is doing a trick on a bike in the sand ',
 'a dog is running through a field ',
 'a man on a dirt bike is being ridden around a hill ',
 'a group of students are standing in front of a tree ',
 'a man and a dog are walking through a grassy field ',
 'a man in a green jacket smiles for the camera ',
 'a young girl eats a bubble ',
 'a boy in a swimsuit is jumping in the water ',
 'a black and white dog is biting a shadow ',
 'a white plane is standing in front of a white car ',
 'a black dog chases a black dog through the grass ',
 'a group of people are playing cricket ',
 'a man and a man bats a break on a park bench ',
 'a man and a man pose for a car ',
 'a dog swims in the water ',
 'a girl in a bikini bathing suit is splashing in a swimming pool ',
 'a little girl is hanging upside down from a metal bar ',
 'a dog jumps over a log in a forest ',
 'a german shepherd is jumping over a pole ',
 'a man in a red jacket and a woman in a red jacket and a woman in a red jacket and a woman in a red jacket and a woman in a red jacket and a woman ',
 'a man is walking down a sidewalk with a woman in a jacket ',
 'a dog is running through a pond ',
 'a dog is gnawing on a snowball in the snow ',
 'a dog is running through a grassy field ',
 'a man plays drums ',
 'a man is demonstrating his picture while another man walks upset ',
 'a dog jumps into a swimming pool ',
 'a man asking up in front of a building ',
 'a man walks past applebee ',
 'a man in a red shirt and helmet is walking through a race ',
 'a man in a red shirt is sitting on a rock ledge ',
 'a boy is riding a tractor on a tractor ',
 'two brown dogs run through a field ',
 'a man in a blue top and jean pants is posing for a picture ',
 'a soccer player in a red uniform is chasing a soccer ball ',
 'a boy in a red shirt is walking on the beach ',
 'a child plays with a net ',
 'a dog is jumping through a grassy field ',
 'a girl in a graduation gown is walking through a field ',
 'a man in a red shirt is sitting on a chair in a chair ',
 'a man and a woman are walking in a cart with handlers in the background ',
 'a man is lying on a pile of sand with a football ',
 'a group of people are gathered outside a building ',
 'a dog is running through the snow ',
 'a shirtless man with a red shirt and a red shirt is sitting on a red and red flag ',
 'a man is sitting on a bench in front of a large tree ',
 'a little girl in a blue shirt is smiling at a slide ',
 'a man jumps over a hurdle ',
 'a woman in a blue shirt is holding a basketball in his hand ',
 'a dog is playing in the water ',
 'a man wearing a red jacket and glasses is smiling ',
 'a small brown and white dog is jumping over a red and white ball ',
 'a group of people are posing for a picture ',
 'a group of people are gathered around a table in a bar ',
 'a boy cringes when drops of people in a boat ',
 'a boy in a red shirt is running through a grassy field ',
 'a man and a man are sitting on a bench in front of a building ',
 'a brown dog is walking in the water ',
 'a woman and a woman sit on a bear slide ',
 'a dog is running through a snowy field ',
 'a dog is running through the snow ',
 'pigeons hot pigeons cage ',
 'a man is training a dog to a black dog to catch a frisbee ',
 'a man with a fishing pole is walking on a beach ',
 'a man with a red hat and beige coat is sitting in a dragon costume ',
 'a dog sits in the car with his mouth open ',
 'a little boy is jumping onto a fence ',
 'a man in a red jacket and black boots sits on a red and white cross street ',
 'a man on a motorcycle lifts his motorcycle ',
 'a man in a leather jacket is sitting in a restaurant ',
 'a man in a red shirt runs through a grassy field ',
 'a man in a yellow jacket is riding a surfboard in the ocean ',
 'a man in a blue jacket is walking through a crowd of people ',
 'a man is rock climbing ',
 'a man and woman sit on a bench amongst trees ',
 'a man is standing on a stage in front of a white building ',
 'a man rides a bike through the woods ',
 'a man in a red shirt is holding a football in his mouth ',
 'a man in a black jacket is holding a beer in his head ',
 'a dog is playing in the ocean with a ball in the background ',
 'a boy in a blue shirt is playing on a red and yellow playground ',
 'a vehicle is driving through a muddy river ',
 'a small dog chases a ball in a grassy field ',
 'a boy swings bat into a baseball game ',
 'a group of people are sitting on a wooden bench ',
 'a man is walking down a windowed stairs ',
 'three children stand on the beach ',
 'a man in a red jacket is walking past a man in a red jacket ',
 'a man and a woman sit on a bench ',
 'a boy in a soccer uniform chasing a soccer ball ',
 'a man in a white shirt and a navy navy navy navy white flowered shirt and a navy navy navy navy white flowered shirt and a navy navy navy smile in the background ',
 'a man and a woman are walking down a sidewalk ',
 'a soccer player in a red shirt is chasing after a soccer ball ',
 'a group of people are sitting around a table in a restaurant ',
 'a dog is running with a snowball in its mouth ',
 'a man is doing a skateboard trick on a bicycle ',
 'a girl in a red jacket is getting splashed with water ']

In [12]:
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)
def calculate_perplexity(generated_stories, tokenizer, model):
    perplexities = []
    with torch.no_grad():
        for story in generated_stories:
            # Tokenize and encode the story
            input_ids = tokenizer.encode(story, return_tensors="pt").to(device)

            # Get the model's predicted probabilities for the tokens
            logits = model(input_ids)[0]

            # Calculate cross entropy loss
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1))

            # Calculate perplexity
            perplexity = torch.exp(loss)
            perplexities.append(perplexity.item())

    avg_perplexity = sum(perplexities) / len(perplexities)
    return avg_perplexity

# Example usage:
avg_perplexity = calculate_perplexity(generated_stories, tokenizer, model)
print("Average Perplexity:", avg_perplexity)


Average Perplexity: 5018.546922442354


In [2]:

from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
model_name = "gpt2"
output_dir = "/kaggle/working/fast"
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)
      
    model = GPT2LMHeadModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

    trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
    )
      
    trainer.train()
    trainer.save_model()

                    
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 3
save_steps = 50000
     
train(
    train_file_path="/kaggle/working/fastcloned.txt",
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

2024-05-08 19:52:49.034371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 19:52:49.034469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 19:52:49.150076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.043300
1000,2.956400
1500,2.911500
2000,2.783000
2500,2.767500
3000,2.767800
3500,2.685900
4000,2.686900
4500,2.685700


In [12]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    ans = (tokenizer.decode(final_outputs[0], skip_special_tokens=True))
    return ans
    
model1_path = "/kaggle/working/fast"
sequence1 = "The man keot the bottle on the table with a girl on the bench ."
max_len = 38
finals = generate_text(model1_path, sequence1, max_len) 


Token indices sequence length is longer than the specified maximum sequence length for this model (10696 > 1024). Running this sequence through the model will result in indexing errors


ValueError: Input length of input_ids is 10696, but `max_length` is set to 38. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [15]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequences, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    finals = []
    n=0
    for sequence in sequences:
        n+=1
        print(n)
        ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
        final_outputs = model.generate(
            ids,
            do_sample=True,
            max_length=max_length,
            pad_token_id=model.config.eos_token_id,
            top_k=50,
            top_p=0.95,
        )
        ans = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
        finals.append(ans)
    return finals
model1_path = "/kaggle/working/fast"
max_len = 40
finals = generate_text(model1_path, gen, max_len) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [1]:
import shutil

# Path to the original .txt file
original_txt_file = "/kaggle/input/outputt/output.txt"

# Path to the desired directory where you want to store the cloned .txt file
desired_directory = "/kaggle/working/fast"
cloned_txt_file = "cloned.txt"

max_lines = 30000

# Counter to keep track of the number of lines copied
lines_copied = 0

# Open the original .txt file in read mode
with open(original_txt_file, 'r', encoding='utf-8') as original_file:
    # Open the cloned .txt file in write mode
    with open(desired_directory + cloned_txt_file, 'w', encoding='utf-8') as cloned_file:
        # Iterate over each line in the original file
        for line in original_file:
            # Write the line to the cloned file
            cloned_file.write(line)
            # Increment the counter
            lines_copied += 1
            # Check if the maximum number of lines has been copied
            if lines_copied >= max_lines:
                break

In [16]:
finals

['a man wearing a hat is firing a gun in a city. Two men arrive at the door with gun, while the third person is a policeman. A policeman tells the man to get out of',
 "a man is walking down a sidewalk with a bicycle in the background  near him. He's suddenly startled by a noise coming from his back. It knocks over the bike with thunder and lightning. A",
 'a smiling girl in a blue jacket and glasses is running in a wooded area  while he is smoking. He is not paying attention when he catches the girl. He notices a sign for a new',
 'a man wearing a black jacket and glasses looks down  into the bar. He decides to go home to visit. He decides to have lunch with his father. His father takes him to the bar and',
 "a man with a brown and white and white jacket is holding a stick  on the dance floor. It's his favorite brand of baseball. He's holding a bag. His friend grabs it with his",
 "a dog jumps over a hurdle  and lands in a mud pit.\nI was playing football in the living room. My mom's 

In [19]:
from nltk.translate.bleu_score import corpus_bleu
# validate with test data

    
# calcuate BLEU score
print("BLEU-1: %f" % corpus_bleu(gen, finals, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(gen, finals, weights=(0.5, 0.5, 0, 0)))

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.101141
BLEU-2: 0.318026
